In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import tensorflow as tf
from typing import NamedTuple, List, Tuple, Sequence,Mapping

import numpy as np

In [60]:
def to_tensors(data:Sequence[Mapping[str,float]], targets:Mapping[str,float]):
    
    vocab = {}
    def to_vector(instance:Mapping[str,float]):
        vector = np.ndarray(len(vocab))
        for key, value in instance.items():
            vector[vocab[key]] = value
        return vector
        
    for instance in data:
        for key,value in instance.items():
            if key not in vocab:
                vocab[key] = len(vocab)
    data_vectors = []
    for instance in data:
        data_vectors.append(to_vector(instance))
        
    data_matrix = np.stack(data_vectors)
    
    target_vector = to_vector(targets)

    print(data_matrix)
    
    return data_matrix, target_vector


[[ 1.  1.]
 [ 0.  1.]]


In [76]:
def estimate_correction_weights(data, targets, reg_lambda = 0.0, debug=False):
    data_matrix, target_vector = to_tensors(data,{"ent_premise_China":1, "ent_hyp_China":1})
    
    instance_weights = tf.Variable(initial_value=tf.zeros([len(data)]))
    data_placeholder = tf.placeholder(tf.float32, shape=data_matrix.shape)
    target_placeholder = tf.placeholder(tf.float32, shape=target_vector.shape)
    total = tf.einsum("ij,j -> i", data_placeholder, instance_weights)
    target_loss = tf.nn.l2_loss(total - target_placeholder)
    regularizer = tf.nn.l2_loss(instance_weights - 1)
    total_loss = target_loss + reg_lambda * regularizer

    sess = tf.Session()
    optimizer = tf.train.AdamOptimizer(0.1)
    opt_op = optimizer.minimize(total_loss)
    project_nonnegative = tf.assign(instance_weights, tf.maximum(0.0,instance_weights))

    sess.run(tf.global_variables_initializer())
    for i in range(0,100):
        feed_dict = {data_placeholder:data_matrix, 
              target_placeholder:target_vector}
        sess.run(opt_op,feed_dict)
        sess.run(project_nonnegative)
        result = sess.run({'total':total,
              'loss':total_loss, 
              'regularizer':regularizer, 
              'weights':instance_weights,
              'target':target_placeholder},feed_dict)
        if debug:
            print(result['loss'])
            print(result['weights'])
            print(result['total'])
    return result['weights'],result['total']

In [77]:
data = [{"ent_premise_China":1, "ent_hyp_China":1},{"ent_premise_China":1, "ent_hyp_China":0}]
targets = {"ent_premise_China":1, "ent_hyp_China":1}

estimate_correction_weights(data, targets, reg_lambda=0.1)

[[ 1.  1.]
 [ 0.  1.]]


(array([ 0.15402147,  0.92377818], dtype=float32),
 array([ 1.07779968,  0.92377818], dtype=float32))